In [2]:
# -*- coding: utf-8 -*-

#module importing
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time



# 2. driver경로설정 및 url주소 설정
driverpath = 'C:/Users/Heewon/driver/chromedriver.exe'
dr = webdriver.Chrome(driverpath)

####

#로그인
dr.get('https://www.kita.net/login/login.do?geturl=http://stat.kita.net/stat/pmsd/china/ChinaWholeList.screen')
dr.find_element_by_id('p_member_id').send_keys('kedkorea12')
dr.find_element_by_id('p_passwd').send_keys('ked2363!')
dr.find_element_by_xpath('''//*[@id="myform"]/fieldset/button''').click()

####

df_tot = pd.DataFrame(columns = ['year','month','country','SC','S_code', 'export', 'import', 'page'])

#한국 URL
naraURL = ['http://stat.kita.net/stat/kts/pum/ItemImpExpList.screen']
nara = ['한국']
for site, 나라 in zip(naraURL, nara) :
    #사이트 진입
    dr.get(site)
    #누계 -> 당월
    dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[4]/select/option[1]''').click()
    #100개씩 보기
    dr.find_element_by_xpath('''//*[@id="listCount"]/option[3]''').click()
    
    #연도 43개
    for year_ in [9, 10, 11, 12, 13, 14, 15, 16, 17] : 
        dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).click()
        년 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).text)
        
        
        #월 12개
        for month_ in range(1,13) : 
            
            if ((년 == 2019) and (month_ == 2)) :
                break;
            dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).click()
            월 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).text)

                
            #조회
            dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[3]/a''').click()                    
            time.sleep(0.5)
            
            abc=0
            temp = 0
            
            while 1:
                
                
                try :
                    if dr.find_element_by_xpath('''//*[@id="mySheet1"]/tbody/tr[3]/td/div/table/tbody/tr/td/div''').text == '조회된 데이터가 없습니다.' :
                        break;
                except :
        
                    try :
                        
                        abc+=1
                        drt = dr.page_source
                        sou = BeautifulSoup(drt,'html.parser')
                        soup = sou.find('tbody')

                        #데이터 적재
                        코드_소 = []
                        소분류 = []
                        수출액 = []
                        수출증감률 = []
                        수입액 = []
                        수입증감률 = []
                        수지 = []
                        
                        
                        #첫페이지 
                        #데이터 적재
                        
                        if abc==1:
                            i=1
#                             print(soup.findAll('td', {'align' : 'Right'})[i*10+5].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2].text)

                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10+5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+6].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10+7].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+8].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10+9].text)
                                i+=1
                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':1})
                            df_tot = pd.concat([df_tot, df])
                        else: 
                            i=1
#                             print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                i+=1

                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':1})
                            df_tot = pd.concat([df_tot, df])
                        print('1page done')
                        ####################
                        if (len(sou.find('div', class_='boardnavi').find('span').findAll('a'))+1) == 1:
                            break;
                        
                        for tmp in range(1, (len(sou.find('div', class_='boardnavi').find('span').findAll('a'))+1)):
                            tmpt = int(dr.find_element_by_xpath('''//*[@id="pageArea"]/span/a[{0}]'''.format(tmp)).text)
                            print(temp, tmpt)
                            if temp > tmpt :
                                slack.chat.post_message('#analysis', '{0}  error'.format(년*100000 +월*1000+tmpt))
                                break;
                            print('{0}  done'.format(년*100000 +월*1000+tmpt))
                            dr.find_element_by_xpath('''//*[@id="pageArea"]/span/a[{0}]'''.format(tmp)).click()
                            time.sleep(0.5)
                            
                            drt = dr.page_source
                            sou = BeautifulSoup(drt,'html.parser')
                            soup = sou.find('tbody')
                            temp+=1
                            #데이터 적재
                            코드_소 = []
                            소분류 = []
                            수출액 = []
                            수출증감률 = []
                            수입액 = []
                            수입증감률 = []
                            수지 = []

                            #데이터 적재
                            i=1
                            print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                i+=1
                            

                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':tmpt})
                            print(df.shape)
                            
                            if df.shape[0] != 100:
                                drt = dr.page_source
                                sou = BeautifulSoup(drt,'html.parser')
                                soup = sou.find('tbody')
                                
                                #데이터 적재
                                코드_소 = []
                                소분류 = []
                                수출액 = []
                                수출증감률 = []
                                수입액 = []
                                수입증감률 = []
                                수지 = []

                                #데이터 적재
                                i=1
                                print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                    코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                    소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                    수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                    #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                    수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                    #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                    #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                    i+=1


                                df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':tmpt})
                                print(df.shape)
                            
                            df_tot = pd.concat([df_tot, df])
                        
                        
                        if temp > tmpt :
                             break;
                        print('done')    
                        #화살표 넘기기
                        dr.find_element_by_xpath('''//*[@id="pageArea"]/a[2]''').click()
                        time.sleep(1)
                        

                    except:
                        break;
            
            print('#analysis', '{0}  done'.format(년*100 +월))
            
            df_tot.to_csv('./{0}korea2.csv'.format(년), index=False)

1page done
0 2
201101002  done
64,901
(100, 8)
1 3
201101003  done
26,502
(100, 8)
2 4
201101004  done
16,387
(100, 8)
3 5
201101005  done
10,464
(100, 8)
4 6
201101006  done
7,342
(100, 8)
5 7
201101007  done
5,359
(100, 8)
6 8
201101008  done
4,144
(100, 8)
7 9
201101009  done
3,340
(100, 8)
8 10
201101010  done
2,628
(100, 8)
done
1page done
9 12
201101012  done
1,766
(100, 8)
10 13
201101013  done
1,492
(100, 8)
11 14
201101014  done
1,266
(100, 8)
12 15
201101015  done
1,080
(100, 8)
13 16
201101016  done
883
(100, 8)
14 17
201101017  done
702
(100, 8)
15 18
201101018  done
574
(100, 8)
16 19
201101019  done
484
(100, 8)
17 20
201101020  done
405
(30, 8)
405
(100, 8)
done
1page done
18 22
201101022  done
274
(100, 8)
19 23
201101023  done
223
(100, 8)
20 24
201101024  done
189
(100, 8)
21 25
201101025  done
154
(100, 8)
22 26
201101026  done
125
(100, 8)
23 27
201101027  done
98
(100, 8)
24 28
201101028  done
79
(100, 8)
25 29
201101029  done
63
(100, 8)
26 30
201101030  done
49
(

1page done
36 42
201105042  done
0
(100, 8)
37 43
201105043  done
0
(100, 8)
38 44
201105044  done
0
(100, 8)
39 45
201105045  done
0
(100, 8)
40 46
201105046  done
0
(100, 8)
41 47
201105047  done
0
(100, 8)
42 48
201105048  done
0
(100, 8)
43 49
201105049  done
0
(100, 8)
44 50
201105050  done
0
(67, 8)
0
(67, 8)
done
1page done
45 41
#analysis 201105  done
1page done
0 2
201106002  done
68,508
(100, 8)
1 3
201106003  done
30,153
(100, 8)
2 4
201106004  done
18,845
(100, 8)
3 5
201106005  done
12,543
(100, 8)
4 6
201106006  done
8,928
(100, 8)
5 7
201106007  done
6,614
(100, 8)
6 8
201106008  done
4,978
(100, 8)
7 9
201106009  done
4,010
(100, 8)
8 10
201106010  done
3,205
(100, 8)
done
1page done
9 12
201106012  done
2,165
(90, 8)
2,165
(100, 8)
10 13
201106013  done
1,805
(100, 8)
11 14
201106014  done
1,491
(100, 8)
12 15
201106015  done
1,255
(100, 8)
13 16
201106016  done
1,030
(100, 8)
14 17
201106017  done
868
(100, 8)
15 18
201106018  done
736
(100, 8)
16 19
201106019  done
5

140
(100, 8)
23 27
201110027  done
112
(100, 8)
24 28
201110028  done
86
(100, 8)
25 29
201110029  done
69
(100, 8)
26 30
201110030  done
53
(100, 8)
done
1page done
27 32
201110032  done
30
(100, 8)
28 33
201110033  done
21
(100, 8)
29 34
201110034  done
16
(100, 8)
30 35
201110035  done
10
(100, 8)
31 36
201110036  done
6
(60, 8)
6
(100, 8)
32 37
201110037  done
3
(100, 8)
33 38
201110038  done
2
(100, 8)
34 39
201110039  done
0
(100, 8)
35 40
201110040  done
0
(100, 8)
done
1page done
36 42
201110042  done
0
(100, 8)
37 43
201110043  done
0
(100, 8)
38 44
201110044  done
0
(30, 8)
0
(100, 8)
39 45
201110045  done
0
(100, 8)
40 46
201110046  done
0
(100, 8)
41 47
201110047  done
0
(60, 8)
0
(100, 8)
42 48
201110048  done
0
(100, 8)
43 49
201110049  done
0
(100, 8)
44 50
201110050  done
0
(95, 8)
0
(95, 8)
done
1page done
45 41
#analysis 201110  done
1page done
0 2
201111002  done
69,307
(100, 8)
1 3
201111003  done
30,938
(100, 8)
2 4
201111004  done
18,940
(100, 8)
3 5
201111005  do

1,106
(100, 8)
12 15
201003015  done
930
(100, 8)
13 16
201003016  done
773
(100, 8)
14 17
201003017  done
657
(100, 8)
15 18
201003018  done
555
(100, 8)
16 19
201003019  done
463
(100, 8)
17 20
201003020  done
383
(100, 8)
done
1page done
18 22
201003022  done
251
(100, 8)
19 23
201003023  done
211
(100, 8)
20 24
201003024  done
175
(100, 8)
21 25
201003025  done
142
(100, 8)
22 26
201003026  done
116
(30, 8)
116
(100, 8)
23 27
201003027  done
92
(100, 8)
24 28
201003028  done
76
(100, 8)
25 29
201003029  done
61
(100, 8)
26 30
201003030  done
47
(100, 8)
done
1page done
27 32
201003032  done
27
(100, 8)
28 33
201003033  done
19
(100, 8)
29 34
201003034  done
14
(100, 8)
30 35
201003035  done
9
(100, 8)
31 36
201003036  done
6
(100, 8)
32 37
201003037  done
3
(100, 8)
33 38
201003038  done
1
(30, 8)
1
(100, 8)
34 39
201003039  done
0
(100, 8)
35 40
201003040  done
0
(100, 8)
done
1page done
36 42
201003042  done
0
(100, 8)
37 43
201003043  done
0
(100, 8)
38 44
201003044  done
0
(100

(100, 8)
1 3
201008003  done
22,969
(100, 8)
2 4
201008004  done
14,782
(100, 8)
3 5
201008005  done
9,595
(100, 8)
4 6
201008006  done
6,493
(100, 8)
5 7
201008007  done
4,801
(100, 8)
6 8
201008008  done
3,642
(100, 8)
7 9
201008009  done
2,832
(100, 8)
8 10
201008010  done
2,364
(100, 8)
done
1page done
9 12
201008012  done
1,591
(100, 8)
10 13
201008013  done
1,296
(100, 8)
11 14
201008014  done
1,092
(90, 8)
1,092
(100, 8)
12 15
201008015  done
911
(100, 8)
13 16
201008016  done
737
(100, 8)
14 17
201008017  done
635
(100, 8)
15 18
201008018  done
519
(100, 8)
16 19
201008019  done
435
(100, 8)
17 20
201008020  done
371
(60, 8)
371
(100, 8)
done
1page done
18 22
201008022  done
259
(100, 8)
19 23
201008023  done
207
(100, 8)
20 24
201008024  done
171
(100, 8)
21 25
201008025  done
142
(100, 8)
22 26
201008026  done
113
(90, 8)
113
(100, 8)
23 27
201008027  done
89
(100, 8)
24 28
201008028  done
72
(100, 8)
25 29
201008029  done
56
(90, 8)
56
(100, 8)
26 30
201008030  done
43
(100,

...

0
(100, 8)
50 53
200810053  done
0
(100, 8)
51 54
200810054  done
0
(100, 8)
52 55
200810055  done
0
(11, 8)
0
(11, 8)
done
1page done
53 51
#analysis 200810  done
1page done
0 2
200811002  done
36,961
(100, 8)
1 3
200811003  done
18,671
(100, 8)
2 4
200811004  done
11,580
(100, 8)
3 5
200811005  done
7,416
(100, 8)
4 6
200811006  done
5,245
(100, 8)
5 7
200811007  done
3,922
(100, 8)
6 8
200811008  done
3,078
(90, 8)
3,078
(100, 8)
7 9
200811009  done
2,412
(100, 8)
8 10
200811010  done
1,894
(100, 8)
done
1page done
9 12
200811012  done
1,270
(100, 8)
10 13
200811013  done
1,047
(100, 8)
11 14
200811014  done
869
(100, 8)
12 15
200811015  done
728
(30, 8)
728
(100, 8)
13 16
200811016  done
622
(90, 8)
622
(100, 8)
14 17
200811017  done
522
(100, 8)
15 18
200811018  done
445
(100, 8)
16 19
200811019  done
372
(100, 8)
17 20
200811020  done
305
(100, 8)
done
1page done
18 22
200811022  done
204
(60, 8)
204
(100, 8)
19 23
200811023  done
165
(100, 8)
20 24
200811024  done
138
(100, 8)
2

48 53
200702053  done
0
(77, 8)
0
(77, 8)
done
1page done
49 51
200702051  done
0
(100, 8)
50 53
200702053  done
0
(77, 8)
0
(77, 8)
done
1page done
51 51
200702051  done
0
(100, 8)
52 53
200702053  done
0
(77, 8)
0
(77, 8)
done
1page done
53 51
#analysis 200702  done
1page done
0 2
200703002  done
42,207
(100, 8)
1 3
200703003  done
20,214
(100, 8)
2 4
200703004  done
12,153
(100, 8)
3 5
200703005  done
7,961
(100, 8)
4 6
200703006  done
5,741
(60, 8)
5,741
(100, 8)
5 7
200703007  done
4,078
(100, 8)
6 8
200703008  done
3,180
(100, 8)
7 9
200703009  done
2,610
(100, 8)
8 10
200703010  done
2,053
(100, 8)
done
1page done
9 12
200703012  done
1,717
(100, 8)
10 13
200703013  done
1,117
(100, 8)
11 14
200703014  done
928
(100, 8)
12 15
200703015  done
806
(100, 8)
13 16
200703016  done
679
(60, 8)
679
(100, 8)
14 17
200703017  done
562
(100, 8)
15 18
200703018  done
473
(100, 8)
16 19
200703019  done
384
(90, 8)
384
(100, 8)
17 20
200703020  done
326
(100, 8)
done
1page done
18 22
2007030

0
(74, 8)
0
(74, 8)
done
1page done
51 51
200706051  done
0
(100, 8)
52 53
200706053  done
0
(100, 8)
53 54
200706054  done
0
(74, 8)
0
(74, 8)
done
1page done
54 51
#analysis 200706  done
1page done
0 2
200707002  done
42,253
(100, 8)
1 3
200707003  done
42,253
(100, 8)
2 4
200707004  done
12,248
(60, 8)
12,248
(100, 8)
3 5
200707005  done
8,284
(100, 8)
4 6
200707006  done
5,905
(100, 8)
5 7
200707007  done
4,442
(30, 8)
4,442
(100, 8)
6 8
200707008  done
3,277
(100, 8)
7 9
200707009  done
2,567
(100, 8)
8 10
200707010  done
2,135
(100, 8)
done
1page done
9 12
200707012  done
1,466
(100, 8)
10 13
200707013  done
1,199
(100, 8)
11 14
200707014  done
1,000
(100, 8)
12 15
200707015  done
845
(100, 8)
13 16
200707016  done
707
(100, 8)
14 17
200707017  done
595
(100, 8)
15 18
200707018  done
486
(100, 8)
16 19
200707019  done
402
(100, 8)
17 20
200707020  done
338
(100, 8)
done
1page done
18 22
200707022  done
235
(100, 8)
19 23
200707023  done
189
(100, 8)
20 24
200707024  done
156
(100

0
(96, 8)
0
(96, 8)
4 52
200711052  done
0
(96, 8)
0
(96, 8)
5 53
200711053  done
0
(96, 8)
0
(96, 8)
done
1page done
6 51
200711051  done
0
(96, 8)
0
(96, 8)
7 52
200711052  done
0
(96, 8)
0
(96, 8)
8 53
200711053  done
0
(96, 8)
0
(96, 8)
done
1page done
9 51
200711051  done
0
(96, 8)
0
(96, 8)
10 52
200711052  done
0
(96, 8)
0
(96, 8)
11 53
200711053  done
0
(96, 8)
0
(96, 8)
done
1page done
12 51
200711051  done
0
(96, 8)
0
(96, 8)
13 52
200711052  done
0
(96, 8)
0
(96, 8)
14 53
200711053  done
0
(96, 8)
0
(96, 8)
done
1page done
15 51
200711051  done
0
(96, 8)
0
(96, 8)
16 52
200711052  done
0
(96, 8)
0
#analysis 200711  done


WebDriverException: Message: chrome not reachable
  (Session info: chrome=72.0.3626.121)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)
